# Dependencias

In [2]:
import numpy as np


# Cargar datos

In [7]:
data = np.load("proyecto_training_data.npy")
print(f"cantidad de registros {data.shape}")

cantidad de registros (1460, 6)


(1460, 6)